In [ ]:
# !pip install PyPDF2
# !pip install ftfy
# !pip install pyresparser
# !pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import spacy
from spacy.matcher import Matcher
import PyPDF2
import os

# Load the Spacy English model
nlp = spacy.load('en_core_web_sm')
import csv
from spacy.matcher import Matcher
import csv

In [ ]:
file_path=r'skills.csv'
with open(file_path, 'r') as file:
    csv_reader = csv.reader(file)
    skills = [row for row in csv_reader]

In [ ]:
skill_patterns = [[{'LOWER': skill}] for skill in skills[0]]

In [ ]:
matcher = Matcher(nlp.vocab)

In [ ]:
for pattern in skill_patterns:
    matcher.add('Skills', [pattern])

In [ ]:
def extract_skills(text):
    doc = nlp(text)
    matches = matcher(doc)
    skills = set()
    for match_id, start, end in matches:
        skill = doc[start:end].text
        skills.add(skill)
    return skills

In [ ]:
def extract_text_from_pdf(file_path:str):
    with open(file_path, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)
        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

In [ ]:
def skills_extractor(file_path):
        # Extract text from PDF
        # path=r'notebook'
        # full_file_path = os.path.join(path, file_path)
        resume_text = extract_text_from_pdf(file_path)

        # Extract skills from resume text
        skills = list(extract_skills(resume_text))
        return skills

In [ ]:
import re
from ftfy import fix_text
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
stopw  = set(stopwords.words('english'))
from pyresparser import ResumeParser
import os
from docx import Document

In [ ]:
# Load dataset:
jd_df=pd.read_csv(r'jd_structured_data.csv')

# Load the extracted resume skills:
file_path=r'CV.pdf'
skills=[]
skills.append(' '.join(word for word in skills_extractor(file_path)))

In [ ]:
def ngrams(string, n=3):
    string = fix_text(string) # fix text
    string = string.encode("ascii", errors="ignore").decode() #remove non ascii chars
    string = string.lower()
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() # normalise case - capital at start of each word
    string = re.sub(' +',' ',string).strip() # get rid of multiple spaces and replace with a single
    string = ' '+ string +' ' # pad names for ngrams...
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [ ]:
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
tfidf = vectorizer.fit_transform(skills)

In [ ]:
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
jd_test = (jd_df['Processed_JD'].values.astype('U'))


In [ ]:
def getNearestN(query):
  queryTFIDF_ = vectorizer.transform(query)
  distances, indices = nbrs.kneighbors(queryTFIDF_)
  return distances, indices

In [ ]:
distances, indices = getNearestN(jd_test)
test = list(jd_test)
matches = []

In [ ]:
for i,j in enumerate(indices):
    dist=round(distances[i][0],2)

    temp = [dist]
    matches.append(temp)

In [ ]:
matches = pd.DataFrame(matches, columns=['Match confidence'])

In [ ]:
jd_df['match']=matches['Match confidence']
jd_df.head(5).sort_values('match')

,Job Title,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Competitors,Average Salary,Average Revenue,Processed_JD,match
4,Data Scientist,2.9,Affinity Solutions,"New York, NY","New York, NY",125.5,1998,Company - Private,Advertising & Marketing,Business Services,"Commerce Signals, Cardlytics, Yodlee",114.5,24319.000761,Data Scientist Affinity Solutions Marketing Cl...,0.71
0,Data Scientist,3.8,Tecolote Research,"Albuquerque, NM","Goleta, CA",750.5,1973,Company - Private,Aerospace & Defense,Aerospace & Defense,-1,72.0,75.000000,"Data Scientist Location: Albuquerque, Educatio...",0.73
2,Data Scientist,4.8,KnowBe4,"Clearwater, FL","Clearwater, FL",750.5,2010,Company - Private,Security Services,Business Services,-1,85.0,300.000000,"KnowBe4, Inc. high growth information security...",0.79
3,Data Scientist,3.8,PNNL,"Richland, WA","Richland, WA",3000.5,1965,Government,Energy,"Oil, Gas, Energy & Utilities","Oak Ridge National Laboratory, National Renewa...",76.5,250500.000000,*Organization Job ID** Job ID: 310709 Director...,0.80
1,Healthcare Data Scientist,3.4,University of Maryland Medical System,"Linthicum, MD","Baltimore, MD",10000.0,1984,Other Organization,Health Care Services & Hospitals,Health Care,-1,87.5,3500.000000,What You Will Do: General Summary The Healthca...,0.86
